<a href="https://colab.research.google.com/github/kayblevision/Multilingual-ITS-using-GPT/blob/main/GPT_answer_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# starting with accessing the API

begin with necessary import statements. Ensure you have your key (don't share it with anyone) in order to access the API.

In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
import openai
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"]='your api key goes here'
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [ ]:
# testing access
response = client.embeddings.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

print(response)

CreateEmbeddingResponse(data=[Embedding(embedding=[0.0022786001209169626, -0.0092921182513237, 0.015702664852142334, -0.007711696904152632, -0.004709530156105757, 0.014915627427399158, -0.00985700823366642, -0.03820939362049103, -0.006931006442755461, -0.028587227687239647, 0.025197889655828476, 0.01813994161784649, -0.003643221454694867, -0.025540631264448166, 0.0005629062652587891, -0.0164389256387949, 0.02833334542810917, 0.005388667341321707, 0.00969198439270258, -0.01641353778541088, -0.015398005023598671, 0.00433187885209918, 0.006956394761800766, -0.0072674015536904335, -0.003890757216140628, 0.01857154257595539, 0.008708187378942966, -0.02267175354063511, 0.011475512757897377, 0.023877698928117752, 0.01562649942934513, -0.003535320982336998, -0.0348835289478302, -0.004214458167552948, -0.026149950921535492, -0.021491197869181633, -0.0057155415415763855, 0.011761130765080452, 0.00844795722514391, 0.004112904891371727, 0.019168168306350708, -0.014407861046493053, 0.00893668271601

# embeddings

Retrieve embeddings for a file you are working with. I'm using a .csv file for this project.

In [ ]:
!pip install openai[embeddings]
!pip install tiktoken
!pip install utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=9c9b9360a4cdc5b474f93aa36e5fdb7cc2d5f12c6305dd4113cc6874b11fd40a
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [ ]:
import pandas as pd
import numpy as np
import tiktoken
import utils
import matplotlib.pyplot as plt

specify which embedding model you want to use-- OpenAI has a few to choose from, linked [here](https://https://platform.openai.com/docs/guides/embeddings)

In [ ]:
def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
embedding_model = "text-embedding-3-large"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191. Choose the right number for you

Loading the data (if using Google Colab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = '/path/to/data/goes/here/data.csv'
df = pd.read_csv(data, header=None)

Obtain embeddings for your text data and save as in a new column in your dataframe.

In [ ]:
import ast  # for converting embeddings saved as strings back to arrays
from scipy import spatial  # for calculating vector similarities for search

In [ ]:
df['embedding'] = df['column_to_receive_embeddings'].apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv('/file/save/path/name.csv', index=False) # save the file so you can access/load it later

In [ ]:
embed = pd.read_csv('/file/save/path/name.csv')
embed.head()

,index,transcriptions,questions,large_embedding
0,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",Who is Dr. Perez Tamayo and what is the topic ...,"[-0.027631612494587898, 0.03299524635076523, -..."
1,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What are some of the cardiac effects that brea...,"[-0.02763524278998375, 0.03302500396966934, -0..."
2,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",Why is it important to inform your primary car...,"[-0.027581216767430305, 0.033021196722984314, ..."
3,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What are some of the side effects of the drugs...,"[-0.02757948823273182, 0.03301912918686867, -0..."
4,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",How can long-term cardiac toxicity be manifest...,"[-0.027606431394815445, 0.033020950853824615, ..."


In [ ]:
embed['embedding'] = embed['embedding'].apply(ast.literal_eval)

# GPT modeling

Define functions to introduce new data to the gpt model for question answering purposes. This includes a search function to rank text based on distance between embeddings, and a query message function to receive a user query, retrieve relevant texts and give a message to GPT.

Reference code and helpful article found [here](https://cookbook.openai.com/examples/question_answering_using_embeddings).

In [ ]:
GPT_MODEL='gpt-3.5-turbo' # you can use this one or one of the newer gpt models

In [ ]:
# search function for the dataset
def strings_ranked_by_relatedness(query: str, df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = client.embeddings.create(
        model=embedding_model,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["column_with_original_text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in embed.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame = embed,
    model: str = GPT_MODEL,
    token_budget: int = 8000,
) -> str:
    """Return a message for GPT."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df2)
    introduction = '''Use the information from the (text containing information) column to answer the subsequent question as succinctly
    as possible. If the answer cannot be found in the data, write "I could not find an answer."'''
    question = f"\n\nQuestion: {query}"
    message = introduction
    return message + question

Now an ask function is defined to return GPT's answer. This code assumes input as a dataframe with relevant text, questions, and embeddings in the same row.

In [ ]:
def ask(
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 8000, # specify your own token budget
    print_message: bool = False,
) -> pd.DataFrame:
    """Answers questions based on (informational text) provided in the same row."""
    responses = []
    for index, row in df.iterrows():
        context = row['column_with_informational_text'] # replace with column names from your dataframe
        question = row['question_column'] # replace with column names from your dataframe

        context_message = query_message(context, model=model, token_budget=token_budget)
        question_message = query_message(question, model=model, token_budget=token_budget)

        if print_message:
            print("Context message:", context_message)
            print("Question message:", question_message)

        messages = [
            {"role": "system", "content": "You answer questions about (relevant topic) based on the (relevant text)."},
            {"role": "user", "content": context_message},
            {"role": "user", "content": question_message}
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0
        )
        response_message = response.choices[-1].message.content  # assuming the last message is the response to the question
        responses.append(response_message)

    df['response'] = responses
    return df

For a dataframe containing questions to be answered, this code gives output of the same dataframe, now with a 'response
column with answers from GPT based on your provided data. However this function can also be adjusted to produce single answers, found at the reference code linked above.

Access the dataframe with answers by calling the ask() function.

In [ ]:
answer = ask()
answer.head(10)

,index,transcriptions,questions,large_embedding,response
0,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",Who is Dr. Perez Tamayo and what is the topic ...,"[-0.027631612494587898, 0.03299524635076523, -...",Dr. Perez Tamayo is a medical professional dis...
1,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What are some of the cardiac effects that brea...,"[-0.02763524278998375, 0.03302500396966934, -0...",Some of the cardiac effects that breast cancer...
2,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",Why is it important to inform your primary car...,"[-0.027581216767430305, 0.033021196722984314, ...",It is important to inform your primary care ph...
3,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What are some of the side effects of the drugs...,"[-0.02757948823273182, 0.03301912918686867, -0...",Some of the side effects of the drugs used for...
4,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",How can long-term cardiac toxicity be manifest...,"[-0.027606431394815445, 0.033020950853824615, ...",Long-term cardiac toxicity can be manifested i...
5,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What steps can be taken to manage the cardiac ...,"[-0.02760116569697857, 0.03298923745751381, -0...",It's important to bring any symptoms like shor...
6,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",Who is Dr. Perez Tamayo and what is the subjec...,"[-0.02759997732937336, 0.0330132320523262, -0....",Dr. Perez Tamayo is a medical professional dis...
7,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What long-term cardiac effects can arise from ...,"[-0.027606071904301643, 0.03302052244544029, -...",Long-term cardiac effects that can arise from ...
8,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",Why is it crucial to notify your primary care ...,"[-0.02763139083981514, 0.03302039951086044, -0...",It is crucial to notify your primary care phys...
9,1,"Hi, this is Dr. Perez Tamayo. Today we're goin...",What are a few examples of side effects caused...,"[-0.027608714997768402, 0.03304910659790039, -...","Shortness of breath, heart palpitations, and s..."


In [ ]:
answer.to_csv('/file/path/here/filename.csv') # save the new dataframe to access it later